<div align="center">

# ArchiMed Images V1.4 - Ultra-High Sensitivity

Enhanced lung segmentation with ultra-sensitive detection for faint lung areas.

## 🔬 Enhanced Features
- **Ultra-low sensitivity thresholds** (0.001 default, down to 0.0001)
- **Enhanced preprocessing** (histogram equalization + gaussian blur)
- **Multiple threshold strategy** with automatic fallbacks
- **Aggressive morphological operations** for better mask cleanup
- **Debug output** for analysis of model probability ranges
- **V1.3 style visualization** with red lung fill/borders, green corner brackets, cyan crop rectangle

</div>

In [ ]:
# Configuration
CSV_FOLDER = "/home/pyuser/data/Paradise_CSV/"
CSV_LABELS_FILE = "Labeled_Data_RAW_Sample.csv"
CSV_SEPARATOR = ";"

DOWNLOAD_PATH = '/home/pyuser/data/Paradise_Test_DICOMs'
IMAGES_PATH = '/home/pyuser/data/Paradise_Test_Images'
MASKS_PATH = '/home/pyuser/data/Paradise_Masks'

TARGET_SIZE = (518, 518)

# Ultra-High Sensitivity Control
# Lower values = more sensitive (detects smaller/fainter lung areas)
# Higher values = less sensitive (only detects clearer lung areas)
# Range: 0.0001 (ultra sensitive) to 0.5 (less sensitive)
MODEL_SENSITIVITY = 0.001

# Enhanced Detection Options
ENABLE_HISTOGRAM_EQUALIZATION = True  # Enhance contrast before segmentation
ENABLE_GAUSSIAN_BLUR = True           # Reduce noise before segmentation
USE_MULTIPLE_THRESHOLDS = True        # Try multiple sensitivity levels
AGGRESSIVE_MORPHOLOGY = True          # More aggressive mask cleanup
ENABLE_DEBUG_OUTPUT = False           # Print segmentation debug info

LUNG_FILL_OPACITY = 0.25
LUNG_BORDER_OPACITY = 0.50

CONVERT = True
SAVE_MASKS = True


In [ ]:
# Dependencies
import ArchiMedConnector.A3_Connector as A3_Conn
import pandas as pd
import os
import pydicom
import numpy as np
from PIL import Image
from tqdm import tqdm
import cv2
import subprocess
import sys

a3conn = A3_Conn.A3_Connector()


In [ ]:
# Segmentation model setup
segmentation_model = None
model_type = None

try:
    import torchxrayvision as xrv
    import torch
    segmentation_model = xrv.baseline_models.chestx_det.PSPNet()
    model_type = 'torchxray'
except ImportError:
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "torchxrayvision"])
        import torchxrayvision as xrv
        import torch
        segmentation_model = xrv.baseline_models.chestx_det.PSPNet()
        model_type = 'torchxray'
    except:
        model_type = 'fallback'


In [ ]:
def segment_lungs(image):
    if model_type == 'torchxray' and segmentation_model is not None:
        return segment_with_torchxray(image)
    else:
        return segment_with_fallback(image)

def enhance_image_preprocessing(image):
    """Enhanced preprocessing for better segmentation detection"""
    if len(image.shape) == 3:
        image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    else:
        image_gray = image.copy()
    
    # Histogram equalization for better contrast
    if ENABLE_HISTOGRAM_EQUALIZATION:
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        image_gray = clahe.apply(image_gray)
    
    # Gaussian blur to reduce noise
    if ENABLE_GAUSSIAN_BLUR:
        image_gray = cv2.GaussianBlur(image_gray, (3, 3), 0)
    
    return image_gray

def segment_with_torchxray(image):
    try:
        # Enhanced preprocessing
        image_preprocessed = enhance_image_preprocessing(image)
        
        image_norm = xrv.datasets.normalize(image_preprocessed, 255)
        image_norm = image_norm[None, ...]
        
        transform = xrv.datasets.XRayResizer(512)
        image_resized = transform(image_norm)
        image_tensor = torch.from_numpy(image_resized).float().unsqueeze(0)
        
        with torch.no_grad():
            output = segmentation_model(image_tensor)
        
        lung_targets = ['Left Lung', 'Right Lung']
        lung_mask = np.zeros((512, 512))
        
        for i, target in enumerate(segmentation_model.targets):
            if target in lung_targets:
                lung_mask += output[0, i].cpu().numpy()
        
        # Debug output
        if ENABLE_DEBUG_OUTPUT:
            print(f"Raw mask stats - Min: {lung_mask.min():.6f}, Max: {lung_mask.max():.6f}, Mean: {lung_mask.mean():.6f}")
        
        lung_mask = cv2.resize(lung_mask, (image.shape[1], image.shape[0]))
        
        # Multiple threshold strategy
        if USE_MULTIPLE_THRESHOLDS:
            thresholds = [MODEL_SENSITIVITY, MODEL_SENSITIVITY * 0.5, MODEL_SENSITIVITY * 0.1, 0.0001]
            best_mask = None
            best_ratio = 0
            
            for threshold in thresholds:
                binary_mask = (lung_mask > threshold).astype(np.uint8)
                lung_ratio = np.sum(binary_mask) / (binary_mask.shape[0] * binary_mask.shape[1])
                
                if ENABLE_DEBUG_OUTPUT:
                    print(f"Threshold {threshold:.6f}: Lung ratio = {lung_ratio:.4f}")
                
                # Target reasonable lung area (2%-40% of image)
                if 0.02 <= lung_ratio <= 0.40:
                    best_mask = binary_mask
                    best_ratio = lung_ratio
                    break
                elif lung_ratio > 0.001 and best_mask is None:  # Accept very small areas as fallback
                    best_mask = binary_mask
                    best_ratio = lung_ratio
            
            if best_mask is not None:
                binary_mask = best_mask
                if ENABLE_DEBUG_OUTPUT:
                    print(f"Selected mask with ratio: {best_ratio:.4f}")
            else:
                binary_mask = (lung_mask > 0.0001).astype(np.uint8)  # Ultra-low fallback
                if ENABLE_DEBUG_OUTPUT:
                    print("Using ultra-low threshold fallback")
        else:
            binary_mask = (lung_mask > MODEL_SENSITIVITY).astype(np.uint8)
        
        # Aggressive morphological operations
        if AGGRESSIVE_MORPHOLOGY:
            kernel_size = 15 if np.sum(binary_mask) > 1000 else 20
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
            binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)
            binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel)
            
            # Fill holes
            contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.fillPoly(binary_mask, contours, 1)
        else:
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
            binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)
            binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel)
        
        return binary_mask
        
    except Exception as e:
        if ENABLE_DEBUG_OUTPUT:
            print(f"TorchXRay segmentation failed: {e}")
        return segment_with_fallback(image)

def segment_with_fallback(image):
    # Enhanced fallback with same preprocessing
    image_preprocessed = enhance_image_preprocessing(image)
    
    _, otsu_mask = cv2.threshold(image_preprocessed, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # More aggressive morphology for fallback
    kernel_size = 25 if AGGRESSIVE_MORPHOLOGY else 20
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    mask_clean = cv2.morphologyEx(otsu_mask, cv2.MORPH_CLOSE, kernel)
    mask_clean = cv2.morphologyEx(mask_clean, cv2.MORPH_OPEN, kernel)
    
    return (mask_clean > 0).astype(np.uint8)


In [ ]:
def create_v13_overlay(image, binary_mask, resize_crop_bounds):
    overlay = image.copy()
    if len(overlay.shape) == 2:
        overlay = cv2.cvtColor(overlay, cv2.COLOR_GRAY2RGB)
    
    resize_y_min, resize_x_min, resize_y_max, resize_x_max = resize_crop_bounds
    img_height, img_width = overlay.shape[:2]
    
    # Darken areas outside the resize crop
    outside_resize_mask = np.ones((img_height, img_width), dtype=bool)
    outside_resize_mask[resize_y_min:resize_y_max, resize_x_min:resize_x_max] = False
    overlay[outside_resize_mask] = (overlay[outside_resize_mask] * 0.5).astype(np.uint8)
    
    # Red lung visualization
    lung_areas = binary_mask > 0
    if np.any(lung_areas):
        # Red fill
        lung_fill_colored = np.zeros_like(overlay)
        lung_fill_colored[lung_areas] = [0, 0, 255]
        fill_opacity = max(LUNG_FILL_OPACITY, 0.4)
        overlay[lung_areas] = cv2.addWeighted(
            overlay[lung_areas], 1.0 - fill_opacity, 
            lung_fill_colored[lung_areas], fill_opacity, 0
        )
        
        # Red border
        lung_mask_uint8 = (binary_mask * 255).astype(np.uint8)
        contours, _ = cv2.findContours(lung_mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        border_mask = np.zeros_like(binary_mask, dtype=np.uint8)
        cv2.drawContours(border_mask, contours, -1, 1, thickness=6)
        
        border_areas = border_mask > 0
        if np.any(border_areas):
            lung_border_colored = np.zeros_like(overlay)
            lung_border_colored[border_areas] = [0, 0, 255]
            border_opacity = max(LUNG_BORDER_OPACITY, 0.6)
            overlay[border_areas] = cv2.addWeighted(
                overlay[border_areas], 1.0 - border_opacity, 
                lung_border_colored[border_areas], border_opacity, 0
            )
    
    # Green corner brackets
    def draw_corner_brackets(img, x1, y1, x2, y2, color, thickness=5, length=60):
        img_height, img_width = img.shape[:2]
        x1 = max(0, min(x1, img_width - 1))
        y1 = max(0, min(y1, img_height - 1))
        x2 = max(0, min(x2, img_width - 1))
        y2 = max(0, min(y2, img_height - 1))
        
        max_length_x = min(length, (x2 - x1) // 4)
        max_length_y = min(length, (y2 - y1) // 4)
        length = max(20, min(max_length_x, max_length_y))
        
        # Top-left
        cv2.line(img, (x1, y1), (min(x1 + length, img_width-1), y1), color, thickness)
        cv2.line(img, (x1, y1), (x1, min(y1 + length, img_height-1)), color, thickness)
        # Top-right
        cv2.line(img, (max(x2 - length, 0), y1), (x2, y1), color, thickness)
        cv2.line(img, (x2, y1), (x2, min(y1 + length, img_height-1)), color, thickness)
        # Bottom-left
        cv2.line(img, (x1, max(y2 - length, 0)), (x1, y2), color, thickness)
        cv2.line(img, (x1, y2), (min(x1 + length, img_width-1), y2), color, thickness)
        # Bottom-right
        cv2.line(img, (x2, max(y2 - length, 0)), (x2, y2), color, thickness)
        cv2.line(img, (max(x2 - length, 0), y2), (x2, y2), color, thickness)
    
    lung_coords = np.where(binary_mask > 0)
    if len(lung_coords[0]) > 0:
        actual_y_min = np.min(lung_coords[0])
        actual_y_max = np.max(lung_coords[0])  
        actual_x_min = np.min(lung_coords[1])
        actual_x_max = np.max(lung_coords[1])
        
        if (actual_x_min >= 0 and actual_y_min >= 0 and 
            actual_x_max < overlay.shape[1] and actual_y_max < overlay.shape[0] and
            actual_x_max > actual_x_min and actual_y_max > actual_y_min):
            draw_corner_brackets(overlay, actual_x_min, actual_y_min, actual_x_max, actual_y_max, (0, 255, 0), 5, 60)
    
    # Cyan rectangle
    cv2.rectangle(overlay, (resize_x_min, resize_y_min), (resize_x_max, resize_y_max), (255, 255, 0), 1)
    
    # Legend
    legend_height = 120
    legend_width = min(700, img_width - 20)
    legend_y_start = img_height - legend_height - 10
    
    legend_background = np.zeros((legend_height, legend_width, 3), dtype=np.uint8)
    legend_area = overlay[legend_y_start:legend_y_start + legend_height, 10:10 + legend_width]
    overlay[legend_y_start:legend_y_start + legend_height, 10:10 + legend_width] = cv2.addWeighted(
        legend_area, 0.25, legend_background, 0.75, 0
    )
    
    text_y = legend_y_start + 20
    cv2.putText(overlay, f"RED = Lung segmentation (Fill: {int(LUNG_FILL_OPACITY*100)}%, Border: {int(LUNG_BORDER_OPACITY*100)}%)", 
               (20, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 180), 2)
    cv2.putText(overlay, "GREEN = Segmentation corner brackets", 
               (20, text_y + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    cv2.putText(overlay, f"CYAN = Final resize crop {TARGET_SIZE[0]}x{TARGET_SIZE[1]}", 
               (20, text_y + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
    
    # Enhanced detection features
    features = []
    if ENABLE_HISTOGRAM_EQUALIZATION: features.append("HistEq")
    if ENABLE_GAUSSIAN_BLUR: features.append("Blur")
    if USE_MULTIPLE_THRESHOLDS: features.append("MultiThresh")
    if AGGRESSIVE_MORPHOLOGY: features.append("AggroMorph")
    features_text = "+".join(features) if features else "Basic"
    
    cv2.putText(overlay, f"Sensitivity: {MODEL_SENSITIVITY} | Enhanced: {features_text}", 
               (20, text_y + 75), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.putText(overlay, f"Model: {model_type}", 
               (20, text_y + 95), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    return overlay

def process_image_with_segmentation(image_array, file_id):
    try:
        binary_mask = segment_lungs(image_array)
        
        # Always save masks and overlays, even for small segmented areas
        if SAVE_MASKS:
            os.makedirs(MASKS_PATH, exist_ok=True)
            
            mask_path = os.path.join(MASKS_PATH, f"{file_id}_mask.png")
            mask_image = (binary_mask * 255).astype(np.uint8)
            cv2.imwrite(mask_path, mask_image)
        
        total_pixels = binary_mask.shape[0] * binary_mask.shape[1]
        lung_pixels = np.sum(binary_mask)
        lung_ratio = lung_pixels / total_pixels
        
        # Only skip cropping for very extreme cases (but still save masks/overlays above)
        if lung_ratio < 0.001 or lung_ratio > 0.98:
            # Create overlay even for extreme cases
            if SAVE_MASKS:
                img_height, img_width = image_array.shape[:2]
                resize_bounds = (0, 0, img_height, img_width)  # Use full image bounds
                overlay_path = os.path.join(MASKS_PATH, f"{file_id}_overlay.png")
                overlay = create_v13_overlay(image_array, binary_mask, resize_bounds)
                cv2.imwrite(overlay_path, overlay)
            return image_array
        
        coords = np.column_stack(np.where(binary_mask > 0))
        if len(coords) == 0:
            # Create overlay even when no coords found
            if SAVE_MASKS:
                img_height, img_width = image_array.shape[:2]
                resize_bounds = (0, 0, img_height, img_width)
                overlay_path = os.path.join(MASKS_PATH, f"{file_id}_overlay.png")
                overlay = create_v13_overlay(image_array, binary_mask, resize_bounds)
                cv2.imwrite(overlay_path, overlay)
            return image_array
        
        y_min, x_min = coords.min(axis=0)
        y_max, x_max = coords.max(axis=0)
        
        img_height, img_width = image_array.shape[:2]
        
        # Generous padding around lungs
        generous_padding = 180
        y_min_padded = max(0, y_min - generous_padding)
        x_min_padded = max(0, x_min - generous_padding)
        y_max_padded = min(img_height, y_max + generous_padding)
        x_max_padded = min(img_width, x_max + generous_padding)
        
        lung_center_y = (y_min_padded + y_max_padded) // 2
        lung_center_x = (x_min_padded + x_max_padded) // 2
        
        target_aspect_ratio = TARGET_SIZE[0] / TARGET_SIZE[1]
        lung_width = x_max_padded - x_min_padded
        lung_height = y_max_padded - y_min_padded
        
        min_width = lung_width + 80
        min_height = lung_height + 80
        
        if min_width / min_height > target_aspect_ratio:
            resize_width = min_width
            resize_height = int(resize_width / target_aspect_ratio)
        else:
            resize_height = min_height
            resize_width = int(resize_height * target_aspect_ratio)
        
        resize_x_min = max(0, lung_center_x - resize_width // 2)
        resize_y_min = max(0, lung_center_y - resize_height // 2)
        resize_x_max = min(img_width, resize_x_min + resize_width)
        resize_y_max = min(img_height, resize_y_min + resize_height)
        
        if resize_x_max == img_width:
            resize_x_min = img_width - resize_width
        if resize_y_max == img_height:
            resize_y_min = img_height - resize_height
            
        resize_x_min = max(0, resize_x_min)
        resize_y_min = max(0, resize_y_min)
        
        # Ensure proper separation between green brackets and cyan rectangle
        lung_coords = np.column_stack(np.where(binary_mask > 0))
        if len(lung_coords) > 0:
            actual_y_min, actual_x_min = lung_coords.min(axis=0)
            actual_y_max, actual_x_max = lung_coords.max(axis=0)
            
            min_separation = 100
            top_sep = actual_y_min - resize_y_min
            left_sep = actual_x_min - resize_x_min
            bottom_sep = resize_y_max - actual_y_max
            right_sep = resize_x_max - actual_x_max
            
            if (top_sep < min_separation or left_sep < min_separation or 
                bottom_sep < min_separation or right_sep < min_separation):
                
                expand_amount = min_separation + 50
                resize_x_min = max(0, actual_x_min - expand_amount)
                resize_y_min = max(0, actual_y_min - expand_amount)
                resize_x_max = min(img_width, actual_x_max + expand_amount)
                resize_y_max = min(img_height, actual_y_max + expand_amount)
        
        # Crop to clear area
        if len(image_array.shape) == 3:
            cropped = image_array[resize_y_min:resize_y_max, resize_x_min:resize_x_max, :]
        else:
            cropped = image_array[resize_y_min:resize_y_max, resize_x_min:resize_x_max]
        
        # Save overlay with proper crop bounds
        if SAVE_MASKS:
            overlay_path = os.path.join(MASKS_PATH, f"{file_id}_overlay.png")
            resize_bounds = (resize_y_min, resize_x_min, resize_y_max, resize_x_max)
            overlay = create_v13_overlay(image_array, binary_mask, resize_bounds)
            cv2.imwrite(overlay_path, overlay)
        
        return cropped
        
    except Exception:
        return image_array


In [ ]:
# Download from ArchiMed
try:
    user_info = a3conn.getUserInfos()
    csv_path = os.path.join(CSV_FOLDER, CSV_LABELS_FILE)
    df = pd.read_csv(csv_path, sep=CSV_SEPARATOR)
    
    file_id_column = None
    for col in ['FileID', 'file_id', 'File_ID']:
        if col in df.columns:
            file_id_column = col
            break
    
    if file_id_column is None:
        raise ValueError("FileID column not found")
    
    file_ids = df[file_id_column].dropna().unique()
    downloaded_files = []
    
    # Create progress bar with total count
    pbar = tqdm(total=len(file_ids), desc="Downloading files", unit="file")
    
    for file_id in file_ids:
        file_id_str = str(file_id)
        dicom_file_path = os.path.join(DOWNLOAD_PATH, f"{file_id}.dcm")
        os.makedirs(DOWNLOAD_PATH, exist_ok=True)
        
        if os.path.exists(dicom_file_path):
            downloaded_files.append(dicom_file_path)
            pbar.update(1)
            continue
        
        try:
            result = a3conn.downloadFile(
                int(file_id_str),
                asStream=False,
                destDir=DOWNLOAD_PATH,
                filename=f"{file_id_str}.dcm",
                inWorklist=False
            )
            
            if result and os.path.exists(dicom_file_path):
                downloaded_files.append(dicom_file_path)
            
            pbar.update(1)
                
        except Exception:
            pbar.update(1)
            pass
    
    pbar.close()
    
except Exception:
    downloaded_files = []


In [ ]:
# Convert DICOM to images
def convert_dicom_to_image(dicom_path, output_path, target_size=TARGET_SIZE):
    try:
        file_id = os.path.splitext(os.path.basename(dicom_path))[0]
        
        dicom_data = pydicom.dcmread(dicom_path)
        image_array = dicom_data.pixel_array
        
        if hasattr(dicom_data, 'PhotometricInterpretation'):
            if dicom_data.PhotometricInterpretation == 'MONOCHROME1':
                image_array = np.max(image_array) - image_array
        
        if image_array.max() > 255:
            image_array = ((image_array - image_array.min()) / 
                          (image_array.max() - image_array.min()) * 255).astype(np.uint8)
        else:
            image_array = image_array.astype(np.uint8)
        
        processed_image = process_image_with_segmentation(image_array, file_id)
        
        if len(processed_image.shape) == 2:
            pil_image = Image.fromarray(processed_image, mode='L')
        else:
            pil_image = Image.fromarray(processed_image)
        
        current_width, current_height = pil_image.size
        target_width, target_height = target_size
        current_ratio = current_width / current_height
        target_ratio = target_width / target_height
        
        if current_ratio > target_ratio:
            new_width = int(current_height * target_ratio)
            new_height = current_height
            left = (current_width - new_width) // 2
            top = 0
            right = left + new_width
            bottom = current_height
        else:
            new_width = current_width
            new_height = int(current_width / target_ratio)
            left = 0
            top = (current_height - new_height) // 2
            right = current_width
            bottom = top + new_height
        
        pil_image = pil_image.crop((left, top, right, bottom))
        pil_image = pil_image.resize(target_size, Image.Resampling.LANCZOS)
        
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        pil_image.save(output_path)
        return True
        
    except Exception:
        return False

# Convert files
if CONVERT and downloaded_files:
    os.makedirs(IMAGES_PATH, exist_ok=True)
    converted_count = 0
    
    with tqdm(total=len(downloaded_files), desc="Converting DICOM files", unit="file") as pbar:
        for dicom_path in downloaded_files:
            file_id = os.path.splitext(os.path.basename(dicom_path))[0]
            output_path = os.path.join(IMAGES_PATH, f"{file_id}.png")
            
            if convert_dicom_to_image(dicom_path, output_path):
                converted_count += 1
            pbar.update(1)
            pbar.set_postfix({"Converted": converted_count})
